<a href="https://colab.research.google.com/github/gauravshetty98/Sentiment-Analysis-Using-LSTMs/blob/main/IMDB_Sentimental_analysis_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score
import pickle
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,Embedding,Flatten
from keras.layers import LSTM, Bidirectional
from keras.layers import Embedding,Flatten,Conv1D,MaxPooling1D
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
df = pd. read_csv('/content/drive/MyDrive/IMDB Dataset.csv')
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [ ]:
df = df.drop(range(30000,50000), axis = 0)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     30000 non-null  object
 1   sentiment  30000 non-null  object
dtypes: object(2)
memory usage: 468.9+ KB


In [ ]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
29995,"New York, I Love You finally makes it to our s...",positive
29996,This movie makes you wish imdb would let you v...,negative
29997,"Space Camp, which had the unfortunate luck to ...",negative
29998,"Octavio Paz, Mexican poet, writer, and diploma...",positive


In [ ]:
X = df.drop('sentiment', axis=1)
y= df['sentiment']
messages = X.copy()
messages.reset_index(inplace=True)
voc_size = 5000

In [ ]:
df.sentiment.replace('positive',1,inplace=True)
df.sentiment.replace('negative',0,inplace=True)
df.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
5,"Probably my all-time favorite movie, a story o...",1
6,I sure would like to see a resurrection of a u...,1
7,"This show was an amazing, fresh & innovative i...",0
8,Encouraged by the positive comments about this...,0
9,If you like original gut wrenching laughter yo...,1


In [ ]:
df.sentiment.value_counts()

1    15015
0    14985
Name: sentiment, dtype: int64

In [ ]:
import re
def clean(text):
    cleaned = re.compile(r'<.*?>')
    return re.sub(cleaned,'',text)

df.review = df.review.apply(clean)
df.review[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows wo

In [ ]:
def is_special(text):
    rem = ''
    for i in text:
        if i.isalnum():
            rem = rem + i
        else:
            rem = rem + ' '
    return rem

df.review = df.review.apply(is_special)
df.review[0]

'One of the other reviewers has mentioned that after watching just 1 Oz episode you ll be hooked  They are right  as this is exactly what happened with me The first thing that struck me about Oz was its brutality and unflinching scenes of violence  which set in right from the word GO  Trust me  this is not a show for the faint hearted or timid  This show pulls no punches with regards to drugs  sex or violence  Its is hardcore  in the classic use of the word It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary  It focuses mainly on Emerald City  an experimental section of the prison where all the cells have glass fronts and face inwards  so privacy is not high on the agenda  Em City is home to many  Aryans  Muslims  gangstas  Latinos  Christians  Italians  Irish and more    so scuffles  death stares  dodgy dealings and shady agreements are never far away I would say the main appeal of the show is due to the fact that it goes where other shows wo

In [ ]:
def to_lower(text):
    return text.lower()

df.review = df.review.apply(to_lower)
df.review[0]

'one of the other reviewers has mentioned that after watching just 1 oz episode you ll be hooked  they are right  as this is exactly what happened with me the first thing that struck me about oz was its brutality and unflinching scenes of violence  which set in right from the word go  trust me  this is not a show for the faint hearted or timid  this show pulls no punches with regards to drugs  sex or violence  its is hardcore  in the classic use of the word it is called oz as that is the nickname given to the oswald maximum security state penitentary  it focuses mainly on emerald city  an experimental section of the prison where all the cells have glass fronts and face inwards  so privacy is not high on the agenda  em city is home to many  aryans  muslims  gangstas  latinos  christians  italians  irish and more    so scuffles  death stares  dodgy dealings and shady agreements are never far away i would say the main appeal of the show is due to the fact that it goes where other shows wo

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

wordnet = WordNetLemmatizer()
corpus = []
for i in range(0, len(messages)):
    lem_review = re.sub('[^a-zA-Z]', ' ', messages['review'][i])
    lem_review = lem_review.lower().split()
    lem_review = [wordnet.lemmatize(word) for word in lem_review if word not in stopwords.words('english')]
    lem_review = ' '.join(lem_review)
    corpus.append(lem_review)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
corpus[0]

'one reviewer mentioned watching oz episode hooked right exactly happened br br first thing struck oz brutality unflinching scene violence set right word go trust show faint hearted timid show pull punch regard drug sex violence hardcore classic use word br br called oz nickname given oswald maximum security state penitentary focus mainly emerald city experimental section prison cell glass front face inwards privacy high agenda em city home many aryan muslim gangsta latino christian italian irish scuffle death stare dodgy dealing shady agreement never far away br br would say main appeal show due fact go show dare forget pretty picture painted mainstream audience forget charm forget romance oz mess around first episode ever saw struck nasty surreal say ready watched developed taste oz got accustomed high level graphic violence violence injustice crooked guard sold nickel inmate kill order get away well mannered middle class inmate turned prison bitch due lack street skill prison experi

In [ ]:
from tensorflow.keras.layers import Embedding, Dropout, Dense, Bidirectional, LSTM
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]

In [ ]:
max_words = max(len(i) for i in onehot_repr)
lst = [i for i in onehot_repr if len(i)==max_words]
max_words

955

In [ ]:
sentence_len = 100
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sentence_len)
embedded_docs[0]

array([3112, 4525,  378, 2257, 3607, 2733, 3438, 1122, 4724, 1213, 1809,
       4230, 3339,   49, 3764, 3175, 1389, 1716, 3454, 2828, 2828, 2769,
       4910, 4124, 2037, 1380, 3340, 3091,  168, 1380,  998, 4592, 2545,
       1988, 1025, 4343, 4010, 4592,   67, 4592, 2060, 4981, 3193, 2007,
       1487, 3328, 2280, 1888, 1780, 3129, 1875, 4910, 4926, 3694, 4919,
       4217, 4981, 2843,  428, 4393, 2971, 1964,  933,  933, 2904, 3540,
       2677, 4083,  204, 2534, 4775, 1181, 4807, 3454, 4705, 4825, 4491,
       4427, 2534, 2544, 3019,  962, 3340, 4919, 2314,  168, 3019, 3790,
       4674, 4981,  841, 3398, 4760,  929, 3429, 2239, 4807, 4460, 4672,
       1393], dtype=int32)

In [ ]:
embedded_vector = 120
model = Sequential()
model.add(Embedding(voc_size, embedded_vector, input_length=sentence_len))
model.add(Bidirectional(LSTM(200, return_sequences=True, input_shape=(120,1))))
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(Bidirectional(LSTM(50)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 120)          600000    
                                                                 
 bidirectional (Bidirectiona  (None, 100, 400)         513600    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 100, 200)         400800    
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 100, 200)         240800    
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 100, 200)         240800    
 nal)                                                   

In [ ]:
X = np.array(df.iloc[:,0].values)
y = np.array(df.sentiment.values)
trainx,testx,trainy,testy = train_test_split(X,y,test_size=0.2,random_state=9)
print("Train shapes : X = {}, y = {}".format(trainx.shape,trainy.shape))
print("Test shapes : X = {}, y = {}".format(testx.shape,testy.shape))

Train shapes : X = (24000,), y = (24000,)
Test shapes : X = (6000,), y = (6000,)


In [ ]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3, random_state=42)

In [ ]:
import tensorflow as tf

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    mode='auto',
    min_delta=0,
    patience=3,
    verbose=0,
    restore_best_weights=True
)

In [ ]:
import os

checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
model.fit(
    X_train,
    y_train,
    epochs=20,
    callbacks=[cp_callback, early_stopping],
    validation_data=(X_test,y_test),
    batch_size=64,
    verbose=1
)

Epoch 1/20
329/329 [==============================] - ETA: 0s - loss: 0.4557 - accuracy: 0.7841
Epoch 1: saving model to training_1/cp.ckpt
329/329 [==============================] - 1122s 3s/step - loss: 0.4557 - accuracy: 0.7841 - val_loss: 0.3936 - val_accuracy: 0.8307
Epoch 2/20
329/329 [==============================] - ETA: 0s - loss: 0.3140 - accuracy: 0.8737
Epoch 2: saving model to training_1/cp.ckpt
329/329 [==============================] - 1106s 3s/step - loss: 0.3140 - accuracy: 0.8737 - val_loss: 0.3744 - val_accuracy: 0.8430
Epoch 3/20
329/329 [==============================] - ETA: 0s - loss: 0.2483 - accuracy: 0.9072
Epoch 3: saving model to training_1/cp.ckpt
329/329 [==============================] - 1110s 3s/step - loss: 0.2483 - accuracy: 0.9072 - val_loss: 0.3872 - val_accuracy: 0.8414
Epoch 4/20
250/329 [=====================>........] - ETA: 3:52 - loss: 0.1908 - accuracy: 0.9324